# Basic Spectrum Generation

The first and most simple way in which TARDIS calculates spectra calculates it directly from the Monte Carlo packets after the final [Monte Carlo Iteration](../montecarlo/index.rst). This simply requires knowledge of the each packet's energy and frequency in the lab frame (see [Reference Frames](../montecarlo/propagation.rst#reference-frames)) at the end of the iteration. The only other quantity needed is the time duration of the simulation $\Delta t$, which is calculated based off of the luminosity of the supernova's photosphere (see [Energy Packet Initialization](../montecarlo/initialization.ipynb)).

<div class="alert alert-info">

Note

The only packets which are used for this calculation are the packets which escape the outer boundary of the
computational domain -- those reabsorbed into the photosphere are not included (see [Packet Propagation](../montecarlo/propagation.rst)).

</div>

The spectrum calculation is very straightforward. A packet of energy $E_\mathrm{packet}$ contributes a
luminosity

$$L_\mathrm{packet} = \frac{E_\mathrm{packet}}{\Delta t}$$

to the spectrum at its frequency.

In the code below, we will see an issue with merely relying on luminosity to give us a spectrum, which will allow us to develop the concept of luminosity *density*, and then correctly plot the TARDIS spectrum.

We start by importing the necessary packages, loading a configuration, and setting up a simulation object (see [Setting up the Simulation](../setup/index.rst)):

In [ ]:
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
from tardis.montecarlo import TARDISSpectrum
from tardis.util.base import quantity_linspace
from astropy import units as u
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
tardis_config = Configuration.from_yaml('tardis_example.yml')

sim = Simulation.from_config(tardis_config)

We now select a number of packets to run through the Monte Carlo simulation, and then run one Monte Carlo iteration (see [Monte Carlo Iteration](../montecarlo/index.rst)):

In [ ]:
N_packets = 5000

# Using the commented out code below, we can also get the number of packets
# from the configuration -- try it out:
#N_packets = tardis_config.no_of_packets

sim.iterate(N_packets)

We call the arrays of each packet's frequency and how much energy the packet has:

In [ ]:
nus = sim.runner.output_nu
nus

In [ ]:
energies = sim.runner.output_energy
energies

Notice that some energies are negative. This means that the packet ended up being reabsorbed into the photosphere (we will separate out these packets later). Also note that the number of elements of our arrays of frequencies and energies is the same as the number of packets that we ran (as it should be):

In [ ]:
len(nus), len(energies)

TARDIS will then calculate a list of the packet luminosities by dividing each element in the array of energies by the time of the simulation $\Delta t$:

In [ ]:
luminosities = energies / sim.runner.time_of_simulation
luminosities

Now, as mentioned before, we only want to include the packets that make it through to the outer boundary. To do this, TARDIS creates an array of booleans (either True or False) called a mask that tells us if the packet should be counted in the spectrum. We then can use that mask to get an array of the frequencies and energies of only the packets which we are interested in:

In [ ]:
emitted_mask = sim.runner.emitted_packet_mask
emitted_mask

In [ ]:
emitted_nus = nus[emitted_mask]
emitted_nus

In [ ]:
emitted_luminosities = luminosities[emitted_mask]
emitted_luminosities

The length of these lists is the number of packets that made it out of the supernova:

In [ ]:
len(emitted_nus), len(emitted_luminosities)

Now, let's plot frequency versus luminosity. We will see a very strange graph, which will lead us into developing a new strategy for plotting the spectrum:

In [ ]:
plt.scatter(emitted_nus, emitted_luminosities)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Luminosity (erg/s)');

This is not the type of plot that we are looking for. We also cannot solve this problem by adding up the luminosities of all packets with the same frequency -- in fact, since frequency is a continuum, it would be extremely unlikely that any two packets have the same *exact* frequency. To solve this problem, we will need the concept of luminosity density.

## Luminosity Density

What we will have to do instead is plot a histogram where we bin up different frequencies that are close to each other into a certain number of bins. We then add up the luminosities of each packet in the bin and divide by the "width" of the bins. For example, if we are plotting between 0 Hz and 50 Hz in 5 bins, we would add up the luminosities of the packets between 0 Hz and 10 Hz, between 10 Hz and 20 Hz, between 20 Hz and 30 Hz, etc., and then divide each value by 10 Hz, which is the width of each bin (note that the bin widths need not be uniform). This will give us the luminosity density with respect to frequency, denoted by $L_\nu$, and measured in ergs per second per Hertz. This can be interpreted as the luminosity per unit Hertz, i.e. how much luminosity will be in an interval with a width of 1 Hz.

The division step here is crucial, as otherwise the values on the y-axis will, for example, approximately double if we double the widths of our bins (as we would be adding about double the luminosity contributions into that bin). We clearly want the values on the y-axis to be independent of the number of bins we break the luminosity into, thus making luminosity density the best way to plot a spectrum.

Note that we can also have luminosity density with respect to wavelength, $L_\lambda$, which we get by binning the packets by wavelength instead of frequency. Since the width of the bins would now have the dimensions of length, $L_\lambda$ will have units of ergs per second per Angstrom (or with meter, nm, cm, etc. instead of Angstrom, depending on that unit you use for wavelength).

Now, to generate our spectrum, we select the bounds for our spectrum and the number of bins that we group the packets into. Feel free to change the number of bins to see how it affects the spectrum!

In [ ]:
# The lowest frequency we plot
freq_start = 1.5e14 * u.Hz

# The highest frequency we plot
freq_stop = 3e15 * u.Hz

# The number of bins
N = 500

The above information can also be gotten from the configuration using the commented-out code below (which is included if you wish to remove the comments and run that cell). Note that the configuration has the bounds specified in terms of wavelengths. Therefore we must convert these to frequencies, noting that the frequency is the speed of light divided by the wavelength (Astropy has a built-in way to do this, which we shall use). Additionally, since wavelength and frequency are inversely related, the lower bound for the wavelengths is the upper bound for the frequencies and vice versa.

In [ ]:
#freq_start = tardis_config.spectrum.stop.to('Hz', u.spectral())

#freq_stop = tardis_config.spectrum.start.to('Hz', u.spectral())

#N = tardis_config.spectrum.num

Next, TARDIS generates the list of frequency bins. The array shown contain the boundaries between successive bins as well as the lower and upper bounds of the spectrum:

In [ ]:
spectrum_frequency = quantity_linspace(freq_start, freq_stop, N+1)
spectrum_frequency

Then, TARDIS creates a histogram where we add up the luminosity in each bin:

In [ ]:
emitted_luminosity_hist = u.Quantity(np.histogram(emitted_nus,
                                                  weights=emitted_luminosities,
                                                  bins=spectrum_frequency.value,
                                                 )[0], "erg / s",)
emitted_luminosity_hist

Finally, we input this information into the TARDISSpectrum class, which will generate the luminosity density with respect to both wavelength and frequency and allow us to plot both of these.

In [ ]:
spectrum = TARDISSpectrum(spectrum_frequency, emitted_luminosity_hist)

In [ ]:
spectrum.plot(mode='frequency')

In [ ]:
spectrum.plot(mode='wavelength')

You may notice that the bins do not appear to be uniformly spaced with respect to wavelength. This is because we use the same bins for both wavelength and frequency, and thus the bins which are uniformly spaced with respect to frequency are not uniformly spaced with respect to wavelength. This is okay though, since luminosity density allows us to alter the bins without significantly altering the graph!

Another thing you may notice in these graphs is the lack of a smooth curve. This is due to **noise**, or the effects of the random nature of Monte Carlo simulations. This makes it very difficult to get a precise spectrum without drastically increasing the number of Monte Carlo packets. To solve these problems, TARDIS uses [virtual packets](virtualpackets.rst) and [the formal integral method](sourceintegration.rst) to generate a spectrum with less noise.